In [1]:
import pyforest
import sys
sys.path.append('/Users/jayceepang/msse/ATOM_CODE/UTILS/')
from RF_functions import *
from GenUtils import *
from dataset import *
import os
import pickle
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE
from sklearn.metrics import accuracy_score, balanced_accuracy_score,precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef

from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
og_neks = ['NEK2_binding','NEK2_inhibition','NEK3_binding','NEK5_binding','NEK9_binding','NEK9_inhibition']
samplings =['none_scaled','UNDER','SMOTE','ADASYN'] 
new_neks=['NEK1', 'NEK11', 'NEK2', 'NEK3', 'NEK4', 'NEK5',
       'NEK6', 'NEK9']

RF_types = ['RF','RF_BCW','BRFC','BRFC_BCW']
og_datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
results_dir='/Users/jayceepang/msse/ATOM_CODE/examples/holdout/holdout_results/'
datapath = '/Users/jayceepang/msse/ATOM_CODE/datasets/holdout_data/featurized/'
rf_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/RF_results/'
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'
test_results=[]
# final_cols=['model','NEK','strategy','feat_type','RF_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
#                 'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy']
feat_cols_dropped =['NEK','compound_id','base_rdkit_smiles', 'subset', 'active'] 

In [3]:
# test_results=[]
# final_cols=['model','NEK','strategy','feat_type','RF_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
#                 'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy']
# for nek in new_neks: 
#     for i in og_neks: 
#         for samp in samplings: 
#             trainX=pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_trainX.csv')
#             train_y =pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_train_y.csv').to_numpy().reshape(-1)
#             testX=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv').drop(columns=feat_cols_dropped)
#             test_y=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv')['active'].to_numpy().reshape(-1)
#             for rf in RF_types: 
#                 model_name = f'{i}_MFP_{samp}_{rf}'
                
#                 print(nek, model_name) 
#                 model = RF_functions.rf_models(trainX, train_y, testX, test_y, rf, {}, True)
#                 # train_df = gather_rf_results(model, trainX, train_y)
#                 test_df = gather_rf_results(model, testX, test_y)
#                 print()

#                 test_df['model'] = f'{nek}_{model_name}'
#                 test_df['NEK'] =nek
#                 test_df['feat_type'] = 'MFP'
#                 test_df['strategy'] = samp
#                 test_df['RF_type'] = rf
#                 test_df['prod_model'] = f'{i}_{samp}_GP_matern'
#                 test_df.to_csv(f'{results_dir}{nek}_{model_name}_holdout_test.csv',index=False) 
#                 test_results.append(test_df.iloc[[0]][final_cols].values.flatten())

#     all_test =  pd.DataFrame(test_results,columns=final_cols)
#     all_test['modeling_type'] = 'RF' 
#     all_test['set'] = 'nanobret_cellular_holdout' 

#     # all_test.to_csv(f'{results_dir}RF/RF_holdout_test_results_all_NEK.csv', index=False)                 
         

In [5]:
test_results=[]
final_cols=['model','prod_model','data NEK','NEK', 'model NEK', 'strategy','feat_type','RF_type',
            'cm','recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy', '1-specificity', 'model strategy',
           ]
for nek in new_neks: 
    testX=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv').drop(columns=feat_cols_dropped)
    test_y=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv')['active'].to_numpy().reshape(-1)
    for i in og_neks: 
        for samp in samplings: 
            trainX=pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_trainX.csv')
            train_y =pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_train_y.csv').to_numpy().reshape(-1)
            
            for rf in RF_types: 
                model_name = f'{i}_MFP_{samp}_{rf}'
                
                print(nek, model_name) 
           
                model = rf_models(trainX, train_y, testX, test_y, rf, {}, True)
         
                test_df = gather_rf_results(model, testX, test_y)
                print()

                test_df['prod_model'] = f'{model_name}'
                test_df['model']=f'{nek}_{model_name}'
                test_df['data NEK'] =nek
                test_df['model NEK'] = i
                test_df['NEK']=nek
                test_df['feat_type'] = 'MFP'
                test_df['strategy'] = samp
                test_df['RF_type'] = rf
                test_df['model strategy'] = f'{samp}_{rf}'
                test_df['1-specificity']=1-test_df['specificity'] 
                
                test_df.to_csv(f'{results_dir}{nek}_{model_name}_holdout_test.csv',index=False) 
                test_results.append(test_df.iloc[[0]][final_cols].values.flatten())

all_test =  pd.DataFrame(test_results,columns=final_cols)
all_test['modeling_type'] = 'RF' 
all_test['set'] = 'nanobret_cellular_holdout' 
all_test.to_csv(f'{results_dir}RF/RF_holdout_test_results_all_NEK.csv', index=False)                 
         

NEK1 NEK2_binding_MFP_none_scaled_RF


NameError: name 'y_true' is not defined

In [ ]:
all_test=pd.read_csv(f'{results_dir}RF/RF_holdout_test_results_all_NEK.csv')
# all_test['prod_model'] = all_test['model'].astype(str).apply(lambda x: '_'.join(x.split('_')[2:]))
# all_test['prod_model'] = all_test['model'].astype(str).apply(lambda x: '_'.join(x.split('_')[1:]))
all_test

In [20]:
# all_test['data NEK'] = all_test['NEK'] 
all_test['model NEK'] = all_test['prod_model'].astype(str).str.split('_').str[0:2].apply(lambda x: '_'.join(x))
all_test=all_test.rename(columns={'NEK':'data NEK','strategy': 'model strategy', 'RF_type':'model version' })
new_order = ['model', 'data NEK', 'prod_model', 'model NEK', 'model version','model strategy', 'feat_type', 'cm', 'recall',
             'specificity', 'accuracy', 'precision', 'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy',
             'set', 'modeling_type']
all_test = all_test[new_order]
all_test.to_csv(f'{results_dir}RF/RF_nanobret_cellular_holdout_results.csv', index=False)

In [19]:
all_test

,model,data NEK,prod_model,model NEK,model version,model strategy,feat_type,cm,recall,specificity,accuracy,precision,f1,ROC_AUC,MCC,balanced_accuracy,set,modeling_type
0,NEK1_NEK2_binding_MFP_none_scaled_RF,NEK1,NEK2_binding_MFP_none_scaled_RF,NEK2_binding,RF,none_scaled,MFP,"[309, 61, 27, 3]",0.100000,0.835135,0.7800,0.046875,0.063830,0.467568,-0.046603,0.467568,nanobret_cellular_holdout,RF
1,NEK1_NEK2_binding_MFP_none_scaled_RF_BCW,NEK1,NEK2_binding_MFP_none_scaled_RF_BCW,NEK2_binding,RF_BCW,none_scaled,MFP,"[310, 60, 28, 2]",0.066667,0.837838,0.7800,0.032258,0.043478,0.452252,-0.069501,0.452252,nanobret_cellular_holdout,RF
2,NEK1_NEK2_binding_MFP_none_scaled_BRFC,NEK1,NEK2_binding_MFP_none_scaled_BRFC,NEK2_binding,BRFC,none_scaled,MFP,"[121, 249, 1, 29]",0.966667,0.327027,0.3750,0.104317,0.188312,0.646847,0.168017,0.646847,nanobret_cellular_holdout,RF
3,NEK1_NEK2_binding_MFP_none_scaled_BRFC_BCW,NEK1,NEK2_binding_MFP_none_scaled_BRFC_BCW,NEK2_binding,BRFC_BCW,none_scaled,MFP,"[312, 58, 25, 5]",0.166667,0.843243,0.7925,0.079365,0.107527,0.504955,0.007165,0.504955,nanobret_cellular_holdout,RF
4,NEK1_NEK2_binding_MFP_UNDER_RF,NEK1,NEK2_binding_MFP_UNDER_RF,NEK2_binding,RF,UNDER,MFP,"[152, 218, 14, 16]",0.533333,0.410811,0.4200,0.068376,0.121212,0.472072,-0.029859,0.472072,nanobret_cellular_holdout,RF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,NEK3_NEK9_inhibition_MFP_SMOTE_BRFC_BCW,NEK3,NEK9_inhibition_MFP_SMOTE_BRFC_BCW,NEK9_inhibition,BRFC_BCW,SMOTE,MFP,"[301, 71, 13, 15]",0.535714,0.809140,0.7900,0.174419,0.263158,0.672427,0.214176,0.672427,nanobret_cellular_holdout,RF
380,NEK3_NEK9_inhibition_MFP_ADASYN_RF,NEK3,NEK9_inhibition_MFP_ADASYN_RF,NEK9_inhibition,RF,ADASYN,MFP,"[333, 39, 28, 0]",0.000000,0.895161,0.8325,0.000000,0.000000,0.447581,-0.090175,0.447581,nanobret_cellular_holdout,RF
381,NEK3_NEK9_inhibition_MFP_ADASYN_RF_BCW,NEK3,NEK9_inhibition_MFP_ADASYN_RF_BCW,NEK9_inhibition,RF_BCW,ADASYN,MFP,"[338, 34, 28, 0]",0.000000,0.908602,0.8450,0.000000,0.000000,0.454301,-0.083619,0.454301,nanobret_cellular_holdout,RF
382,NEK3_NEK9_inhibition_MFP_ADASYN_BRFC,NEK3,NEK9_inhibition_MFP_ADASYN_BRFC,NEK9_inhibition,BRFC,ADASYN,MFP,"[333, 39, 28, 0]",0.000000,0.895161,0.8325,0.000000,0.000000,0.447581,-0.090175,0.447581,nanobret_cellular_holdout,RF


In [21]:
df_check = pd.read_csv('/Users/jayceepang/msse/ATOM_CODE/examples/holdout/holdout_results/all_nanobret_cellular_holdout_results.csv')

In [24]:
print(df_check['data NEK'].value_counts())
nek1 = df_check[df_check['data NEK']=='NEK1']

data NEK
NEK1     120
NEK11    120
NEK2     120
NEK3     120
NEK9      24
NEK5      24
NEK4      24
NEK6      24
Name: count, dtype: int64
